In [12]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
pyspark.__version__

'3.5.4'

In [3]:
#1 Question 1: Install Spark and PySpark
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 17:42:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.5.4'

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-05 17:43:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.222, 18.155.128.6, 18.155.128.46, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   184MB/s    in 0.3s    

2025-03-05 17:43:26 (184 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [29]:
# Question 2: Yellow October 2024

df = spark.read.parquet('yellow_tripdata_2024-10.parquet').repartition(4)

In [7]:
df.write.parquet('hw/')

In [9]:
!ls -lh hw

total 90M
-rw-r--r-- 1 leo leo   0 Mar  5 17:49 _SUCCESS
-rw-r--r-- 1 leo leo 23M Mar  5 17:49 part-00000-2f1e1d06-195b-4637-8005-4f2beeaf27f5-c000.snappy.parquet
-rw-r--r-- 1 leo leo 23M Mar  5 17:49 part-00001-2f1e1d06-195b-4637-8005-4f2beeaf27f5-c000.snappy.parquet
-rw-r--r-- 1 leo leo 23M Mar  5 17:49 part-00002-2f1e1d06-195b-4637-8005-4f2beeaf27f5-c000.snappy.parquet
-rw-r--r-- 1 leo leo 23M Mar  5 17:49 part-00003-2f1e1d06-195b-4637-8005-4f2beeaf27f5-c000.snappy.parquet


In [ ]:
# so they are approximately 25MB

In [11]:
# Question 3: Count records

df.select('tpep_pickup_datetime').show()

+--------------------+
|tpep_pickup_datetime|
+--------------------+
| 2024-10-01 22:46:41|
| 2024-10-09 22:11:58|
| 2024-10-03 08:41:40|
| 2024-10-09 13:56:51|
| 2024-10-09 10:36:09|
| 2024-10-08 12:44:13|
| 2024-10-02 20:19:21|
| 2024-10-02 12:04:58|
| 2024-10-02 20:40:44|
| 2024-10-02 17:09:58|
| 2024-10-05 02:59:55|
| 2024-10-01 23:00:00|
| 2024-10-03 23:50:44|
| 2024-10-08 10:26:57|
| 2024-10-06 12:03:45|
| 2024-10-10 16:17:27|
| 2024-10-02 18:41:49|
| 2024-10-03 13:35:04|
| 2024-10-02 16:31:40|
| 2024-10-09 20:33:58|
+--------------------+
only showing top 20 rows



In [25]:
df.select('tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','DOLocationID')\
    .withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime))\
    .filter(F.col('pickup_date') == '2024-10-15').count()

128893

In [26]:
# closest answer is: 125,567

In [27]:
# Question 4: Longest trip

df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [39]:
longest_trip = df.select('tpep_pickup_datetime','tpep_dropoff_datetime') \
    .withColumn('trip_time_seconds', F.unix_timestamp('tpep_dropoff_datetime') - F.unix_timestamp('tpep_pickup_datetime') ) \
    .withColumn('trip_time_hours', F.col('trip_time_seconds')/3600) \
    .agg({'trip_time_hours': 'max'}) \
    .collect()[0][0]

print(f'longest_trip: {longest_trip}')

longest_trip: 162.61777777777777


In [46]:
# another option:

longest_trip = df.select('tpep_pickup_datetime','tpep_dropoff_datetime') \
    .withColumn('trip_time_seconds', F.unix_timestamp('tpep_dropoff_datetime') - F.unix_timestamp('tpep_pickup_datetime') ) \
    .withColumn('trip_time_hours', F.col('trip_time_seconds')/3600) \
    .groupby() \
    .max('trip_time_hours') \
    .collect()[0].asDict()

print(f'{longest_trip}')

{'max(trip_time_hours)': 162.61777777777777}


In [47]:
# 4 User interface
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-05 18:22:42--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.187, 18.155.128.222, 18.155.128.46, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-05 18:22:42 (238 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [50]:
df_zone = spark.read.option("header","true").csv('taxi_zone_lookup.csv')

In [51]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [56]:
df_join = df.withColumnRenamed('PULocationID','LocationID') \
        .join(df_zone, on=['LocationID'], how='outer') \
        .select('LocationID','Borough','Zone','service_zone','tpep_pickup_datetime','tpep_dropoff_datetime')

In [57]:
df_join.show()

+----------+-------+--------------+------------+--------------------+---------------------+
|LocationID|Borough|          Zone|service_zone|tpep_pickup_datetime|tpep_dropoff_datetime|
+----------+-------+--------------+------------+--------------------+---------------------+
|         1|    EWR|Newark Airport|         EWR| 2024-10-05 14:21:33|  2024-10-05 14:21:43|
|         1|    EWR|Newark Airport|         EWR| 2024-10-03 14:25:48|  2024-10-03 14:25:55|
|         1|    EWR|Newark Airport|         EWR| 2024-10-03 13:56:39|  2024-10-03 13:56:44|
|         1|    EWR|Newark Airport|         EWR| 2024-10-01 14:20:00|  2024-10-01 14:20:08|
|         1|    EWR|Newark Airport|         EWR| 2024-10-06 12:56:08|  2024-10-06 12:56:14|
|         1|    EWR|Newark Airport|         EWR| 2024-10-03 00:03:40|  2024-10-03 00:03:45|
|         1|    EWR|Newark Airport|         EWR| 2024-10-02 18:21:24|  2024-10-02 18:22:21|
|         1|    EWR|Newark Airport|         EWR| 2024-10-07 13:20:55|  2024-10-0

In [59]:
# show the least frequent pickup zone
df_join.groupby('Zone').count().orderBy(F.asc('count')).show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|     Freshkills Park|    1|
|    Great Kills Park|    1|
|       Rikers Island|    2|
|       Arden Heights|    2|
|Governor's Island...|    3|
|         Jamaica Bay|    3|
| Green-Wood Cemetery|    3|
|Eltingville/Annad...|    4|
|   Rossville/Woodrow|    4|
|       West Brighton|    4|
|       Port Richmond|    4|
|Charleston/Totten...|    4|
|         Great Kills|    6|
|        Crotona Park|    6|
|     Mariners Harbor|    7|
|Heartland Village...|    7|
|Saint George/New ...|    9|
|             Oakwood|    9|
|       Broad Channel|   10|
|New Dorp/Midland ...|   10|
+--------------------+-----+
only showing top 20 rows



In [64]:
# these seem to be invalid, let's filter bad data
df_join.filter(F.col('Zone').isin('Freshkills Park','Great Kills Park') ).show()

+----------+-------------+----------------+------------+--------------------+---------------------+
|LocationID|      Borough|            Zone|service_zone|tpep_pickup_datetime|tpep_dropoff_datetime|
+----------+-------------+----------------+------------+--------------------+---------------------+
|        99|Staten Island| Freshkills Park|   Boro Zone|                NULL|                 NULL|
|       110|Staten Island|Great Kills Park|   Boro Zone|                NULL|                 NULL|
+----------+-------------+----------------+------------+--------------------+---------------------+



In [67]:
df_join.filter(F.col('tpep_pickup_datetime').isNotNull() & F.col('tpep_dropoff_datetime').isNotNull() ) \
    .groupby('Zone') \
    .count() \
    .orderBy(F.asc('count')) \
    .limit(10) \
    .show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
|       Arden Heights|    2|
|       Rikers Island|    2|
|         Jamaica Bay|    3|
| Green-Wood Cemetery|    3|
|Charleston/Totten...|    4|
|   Rossville/Woodrow|    4|
|       West Brighton|    4|
|       Port Richmond|    4|
|Eltingville/Annad...|    4|
|         Great Kills|    6|
|        Crotona Park|    6|
|     Mariners Harbor|    7|
|Heartland Village...|    7|
|Saint George/New ...|    9|
|             Oakwood|    9|
|New Dorp/Midland ...|   10|
|       Broad Channel|   10|
|         Westerleigh|   12|
|     Pelham Bay Park|   12|
+--------------------+-----+
only showing top 20 rows

